In [ ]:
import requests
from bs4 import BeautifulSoup
import re

def find_sources(headline, sources):
    """
    Finds three sources for the given headline using Google Search.

    Args:
        headline: The headline to search for.
        sources: A list of preferred news sources to prioritize.

    Returns:
        A list of three source URLs, or an empty list if no sources are found.
    """

    search_query = f'"{headline}"'
    for source in sources:
        search_query += f' site:{source}'

    try:
        response = requests.get(f'https://www.google.com/search?q={search_query}')
        response.raise_for_status()  # Raise an exception for bad status codes

        soup = BeautifulSoup(response.content, 'html.parser')

        # Extract URLs from search results
        urls = []
        for result in soup.find_all('a', href=re.compile(r'^https?://')):
            url = result['href']
            if url.startswith('http') and not url.startswith('http://www.google.com'):
                urls.append(url)

        # Prioritize URLs from preferred sources
        prioritized_urls = [url for url in urls if any(source in url for source in sources)]

        return prioritized_urls[:3]  # Return the first three prioritized URLs
    except requests.exceptions.RequestException as e:
        print(f"Error fetching search results: {e}")
        return []

# Example usage
headline = "AI breakthrough in medical imaging"
preferred_sources = ["nytimes.com", "bbc.com", "nature.com"]

source_urls = find_sources(headline, preferred_sources)
print(source_urls)